http://www.tensorfly.cn/tfdoc/tutorials/mnist_beginners.html

### 1、导入数据

In [5]:
import input_data

In [6]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### 2. 查看数据

查看训练集数据

In [30]:
mnist.train.images.shape

(55000, 784)

In [7]:
mnist.train.images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

查看训练集标签

In [31]:
mnist.train.labels.shape

(55000, 10)

In [8]:
mnist.train.labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

### 3.定义权重和偏置

导入 tensorflow

In [9]:
import tensorflow as tf

定义训练数据，一维长度为 None 表示任意长度，二维长度 784 表示每个图片有 28 * 28 像素组成

In [10]:
x = tf.placeholder("float", [None, 784])

In [32]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>

定义权重 W 并初始化，维度是[784，10]，因为我们想要用 784 维的图片向量乘以它以得到一个10维的证据值向量，每一位对应不同数字类

In [11]:
W = tf.Variable(tf.zeros([784,10]))

Instructions for updating:
Colocations handled automatically by placer.


In [33]:
W

<tf.Variable 'Variable:0' shape=(784, 10) dtype=float32_ref>

定义偏置 b，形状是[10]，所以我们可以直接把它加到输出上面

In [12]:
b = tf.Variable(tf.zeros([10]))

In [34]:
b

<tf.Variable 'Variable_1:0' shape=(10,) dtype=float32_ref>

### 4. 定义模型

证据提取公式是线性模型 `xW+b`，激活函数使用 softmax，对线性结果进行多结果分类

y 表示我们的预测结果

In [35]:
y = tf.nn.softmax(tf.matmul(x,W) + b)
y

<tf.Tensor 'Softmax_1:0' shape=(?, 10) dtype=float32>

### 5. 定义损失函数

定义训练数据的真实结果，一维长度任意，二维长度 10 表示目标分类 0-9

In [14]:
y_ = tf.placeholder("float", [None,10])

定义交叉熵为损失函数

In [15]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

首先，用 tf.log 计算 y 的每个元素的对数。接下来，我们把 y_ 的每一个元素和 tf.log(y_) 的对应元素相乘。最后，用 tf.reduce_sum 计算张量的所有元素的总和。

### 6. 定义优化器

TensorFlow 拥有一张描述你各个计算单元的图，它可以自动地使用反向传播算法(backpropagation algorithm)来有效地确定你的变量是如何影响你想要最小化的那个成本值的。然后，TensorFlow会用你选择的优化算法来不断地修改变量以降低成本。

In [16]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

用梯度下降算法（gradient descent algorithm）以0.01的学习速率最小化交叉熵。梯度下降算法（gradient descent algorithm）是一个简单的学习过程，TensorFlow只需将每个变量一点点地往使成本不断降低的方向移动。

### 7. 初始化工作

初始化变量

In [17]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


启动图

In [18]:
sess = tf.Session()
sess.run(init)

### 8. 进行训练

该循环的每个步骤中，我们都会随机抓取训练数据中的100个批处理数据点，然后我们用这些数据点作为参数替换之前的占位符来运行train_step。

In [22]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

feed_dict 里 x 是本次随机抽取的训练数据，y_ 是对应的真实标签，训练时它们会替换之前的占位符来运行train_step

### 9. 评估模型

对比预测结果和真实结果

In [19]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

把 bool 值转换为 float 后取平均值

In [20]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

使用测试数据集计算最终的准确率

In [23]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9185
